<h1> Eyes of Hajj </h1>
<h3>Hajj Crowd Abnormal Behavior Detection</h3>
<h3>Project Workflow</h3>
<ol>
    <li>Loading the HAJJv2 Dataset.</li>
    <li>Extracting the images (frames) from the videos and merge the labesls into a single file for train & test.</li>
    <li>Transforming the data to Yolo format where we optain a text file (label) for each image in train & test & val. </li>
    <li>Changeing the Yolo feature extractor's backbone to MobileNet/ResNet50 to optimize the feature extraction.</li>
    <li>Applaying the Lucas-Kanade algorithm on the frames (images) from the original dataset to estimate the optical flow (Magnitude & Orientation/Direction).</li>
    <li>Taking the mean & varience & STD of the optical flow and feed it into the Random Forest classifier to predect the actual class of the abnormal behavior.</li>
    <li>Using the Kalman filter with the RF predections to optimaize the Yolo's object tracking.</li>
    <li>Evaluating the models' performences on the testing data nad save the output in a new folder.</li>
</ol>

In [ ]:
# Importing the required libraries

import pandas as pd
import numpy as np
import os
import cv2

### 8. Evaluating the models' performences on the testing data:

In [ ]:
# Function to convert video to frames and store them in a list
def video_to_frames(video_path):
    # Open the video file
    video_capture = cv2.VideoCapture(video_path)

    # Check if the video file is opened successfully
    if not video_capture.isOpened():
        print("Error: Could not open video file.")
        return

    frames = []

    # Loop through the video frames
    while True:
        ret, frame = video_capture.read()

        # Break the loop if we've reached the end of the video
        if not ret:
            break

        # Resize the frame to 640x640
        frame = cv2.resize(frame, (640, 640))

        frames.append(frame)

    # Release the video capture object
    video_capture.release()

    return frames

In [ ]:
def parse_bboxes(bbox_str):
    try:
        # Evaluate the string while providing safe globals and locals to prevent potential security issues
        bbox_list = eval(bbox_str, {"__builtins__": None}, {})
        parsed_bboxes = []

        for bbox_data in bbox_list:
            if len(bbox_data) == 5:
                bbox = np.array(bbox_data[:4])
                confidence = float(bbox_data[4])
                parsed_bboxes.append((bbox, confidence))
            else:
                print(f"Ignoring invalid bounding box data: {bbox_data}")

        return parsed_bboxes
    except Exception as e:
        print(f"Error parsing bounding boxes: {e}")
        return []

# Function to display and save annotated frames
def save_annotated_frames(frames, annotations, output_dir):
    os.makedirs(output_dir, exist_ok=True)

    for i, annotation in annotations.iterrows():
        frame = frames[i]
        frame_number = annotation['Frame_Number']
        bboxes_str = annotation['Bboxes']
        bboxes = parse_bboxes(bboxes_str)

        # Create a copy of the frame to draw bounding boxes
        annotated_frame = frame.copy()

        for bbox, confidence in bboxes:
            x1, y1, x2, y2 = bbox.astype(int)
            cv2.rectangle(annotated_frame, (x1, y1), (x2, y2), (100, 0, 255), 2)  # Light red with lower red channel value
            cv2.putText(annotated_frame, f'Confidence: {confidence:.2f}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (100, 0, 255), 2)

        # Save the annotated frame
        output_path = os.path.join(output_dir, f'frame_{frame_number}.jpg')
        cv2.imwrite(output_path, annotated_frame)

# Load the DataFrame from a CSV file (replace 'annotations.csv' with your file path)
annotations_df = pd.read_csv('annotations.csv')

video_path = "./HAJJv2_Dataset/Original_Data/Test/Videos/9.mp4"  # Replace with the path to your video file
frames = video_to_frames(video_path)

# Specify the output directory for saving frames
output_directory = "./Model_Testing_Results"

# Create the output directory if it doesn't exist
os.makedirs(output_directory, exist_ok=True)

# Call the save function with your frames, annotations, and output directory
save_annotated_frames(frames, annotations_df, output_directory)